In [111]:
# https://www.properati.com.ar/data
#! wget -O ar_properties.csv.gz -q https://storage.googleapis.com/properati-data-public/ar_properties.csv.gz
#! gzip -d -f ./ar_properties.csv.gz

In [112]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

import datetime

import warnings
warnings.filterwarnings('ignore')

import re

In [113]:
pd.options.display.max_colwidth = -1

# Lectura de datos

In [114]:
#df_properties = pd.read_csv('./ar_properties.csv', index_col="id")
df_apred = pd.read_csv('../archivos/a_predecir.csv', index_col="id")

In [115]:
#df = df_properties.copy()
df_pred = df_apred.copy()

In [116]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1087 entries, 1 to 1087
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ad_type          1087 non-null   object 
 1   start_date       1087 non-null   object 
 2   end_date         1087 non-null   object 
 3   created_on       1087 non-null   object 
 4   lat              1087 non-null   float64
 5   lon              1087 non-null   float64
 6   l1               1087 non-null   object 
 7   l2               1087 non-null   object 
 8   l3               1087 non-null   object 
 9   rooms            1083 non-null   float64
 10  bedrooms         958 non-null    float64
 11  bathrooms        1083 non-null   float64
 12  surface_total    1087 non-null   float64
 13  surface_covered  1087 non-null   float64
 14  price            0 non-null      float64
 15  currency         1087 non-null   object 
 16  price_period     0 non-null      float64
 17  title         

# Análisis del conjunto de datos

In [117]:
#analizamos los valores de las distintas variables
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1087 entries, 1 to 1087
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ad_type          1087 non-null   object 
 1   start_date       1087 non-null   object 
 2   end_date         1087 non-null   object 
 3   created_on       1087 non-null   object 
 4   lat              1087 non-null   float64
 5   lon              1087 non-null   float64
 6   l1               1087 non-null   object 
 7   l2               1087 non-null   object 
 8   l3               1087 non-null   object 
 9   rooms            1083 non-null   float64
 10  bedrooms         958 non-null    float64
 11  bathrooms        1083 non-null   float64
 12  surface_total    1087 non-null   float64
 13  surface_covered  1087 non-null   float64
 14  price            0 non-null      float64
 15  currency         1087 non-null   object 
 16  price_period     0 non-null      float64
 17  title         

In [118]:
df_pred.loc[df_apred.rooms.isna(),"rooms"] = df_pred.loc[df_apred.rooms.isna(),"bedrooms"]

In [119]:
df_pred.loc[1042,"rooms"] = 1

In [120]:
df_pred[["bathrooms","surface_covered"]].groupby("bathrooms").mean()

,surface_covered
bathrooms,
0.0,253.030000
1.0,58.869472
2.0,129.872227
3.0,215.594928
4.0,282.867895
5.0,290.666667
7.0,551.520000


In [121]:
df_pred.loc[1080,["rooms","bedrooms","bathrooms"]] = [3,2,1]

In [122]:
df_pred.loc[df_pred.bedrooms.isna(),"bedrooms"] = df_pred.loc[df_pred.bedrooms.isna(),"rooms"]

In [123]:
df_pred.loc[877,["bathrooms"]] = [2]
df_pred.loc[270,["bathrooms"]] = [2]
df_pred.loc[356,["bathrooms"]] = [2]

In [124]:
df_pred.loc[df_pred.description.isna(), "description"] = df_pred.loc[df_pred.description.isna(), "title"]
df.loc[df.description.isna(), "description"] = df.loc[df.description.isna(), "title"]

In [125]:
df_pred["avenida"] = 0
df["avenida"] = 0

In [126]:
df_pred.loc[df_pred.loc[df_pred.title.str.lower().str.contains("(?:avenida|av)")].index,"avenida"] = 1
df.loc[df.loc[df.title.str.lower().str.contains("(?:avenida|av)")].index,"avenida"] = 1

In [ ]:
df_pred["profesionales"] = 0
df["profesionales"] = 0

In [ ]:
df_pred.loc[df_pred.loc[df_pred.description.str.contains("apto profe[cs]iona[l|es]")].index,["profesionales"]] = 1
df.loc[df.loc[df.description.str.contains("apto profe[cs]iona[l|es]")].index,["profesionales"]] = 1

In [138]:
patron= "((?:avenida|av)\s+(\w+)+\s+(\w+))"
df_pred["calles"] = df_pred.title.apply(lambda texto: re.search(patron,texto.lower()).group() if type(re.search(patron,texto.lower())) == re.Match else "null")

In [139]:
df_pred.calles.value_counts()

null                      1065
avenida cabildo al        4   
avenida las heras         2   
avenida alvarez thomas    1   
avenida pavon al          1   
avenida corrientes al     1   
av cabildo 4967           1   
av alvear al              1   
avenida callao al         1   
av cramer al              1   
av luis maria             1   
avenida monroe al         1   
avenida rivadavia al      1   
av corrientes al          1   
avenida belgrano 2960     1   
avenida coronel niceto    1   
av triunvirato 3800       1   
avenida balbin al         1   
avenida cordoba 2800      1   
Name: calles, dtype: int64

In [55]:
df_pred.loc[df_pred.title.str.lower().str.contains("[a-z] al")].index

Int64Index([   2,    3,    5,    6,    7,    9,   10,   15,   16,   20,
            ...
            1076, 1078, 1079, 1080, 1081, 1083, 1084, 1085, 1086, 1087],
           dtype='int64', name='id', length=666)

In [ ]:
df_pred.loc[df_pred.loc[df_pred.description.str.contains("apto profe[cs]iona[l|es]")].index,["profesionales"]] = 1

In [43]:
df_pred.loc[df_pred.description.isna()]

,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,rooms,...,title,description,property_type,operation_type,details,additionals,services,l4,l5,l6
id,,,,,,,,,,,,,,,,,,,,,
467,Propiedad,2022-04-01,2022-04-01,2022-04-01,-58.451284,-34.552775,Argentina,Capital Federal,Belgrano,1.0,...,"Av. Del Libertador Al 6400 - Departamento en Venta en Belgrano, CABA de 37.25 m2, 1 baño, a .",NaN,Departamento,Venta,"Superficie 37.25m2, Ambientes 1, Baños 1, Sup. Cubierta 33.8m2",SUM,Luz,NaN,NaN,NaN
617,Propiedad,2022-04-01,2022-04-01,2022-04-01,-58.419221,-34.630538,Argentina,Capital Federal,Boedo,3.0,...,"Avenida Pavon Al 4300 - Departamento en Venta en Boedo, CABA de 64.5 m2 con 2 dormitorios, 1 baño, a .",NaN,Departamento,Venta,"Superficie 64.5m2, Ambientes 3, Baños 1, Dormitorios 2, Sup. Cubierta 64.5m2, Antigüedad50 años, Cantidad de pisos 4",Aire Acondicionado,"Gas, Luz",NaN,NaN,NaN
1055,Propiedad,2022-04-01,2022-04-01,2022-04-01,-65.217590,-26.808285,Argentina,Capital Federal,San Nicolás,2.0,...,"Tucuman Al 700 - Departamento en Venta en San Nicolás, CABA de 35.29 m2 con 1 dormitorio, 1 baño, a .",NaN,Departamento,Venta,"Superficie 35.29m2, Ambientes 2, Baños 1, Dormitorios 1, Sup. Cubierta 35.29m2","Aire Acondicionado, SUM, Gimnasio, Terraza",Luz,NaN,NaN,NaN
1080,Propiedad,2022-04-01,2022-04-01,2022-04-01,-58.460145,-34.637484,Argentina,Capital Federal,Flores,3.0,...,"Cuenca Al 800 - Departamento en Venta en Flores, CABA de 65 m2, a .",NaN,Departamento,Venta,"Superficie 65m2, Sup. Cubierta 60m2",NaN,NaN,NaN,NaN,NaN


# Tratamiento de nulls y outliers

In [430]:
df = df.loc[df["l1"] == "Argentina"]

In [431]:
df = df.loc[df["l2"] == "Capital Federal"]

In [449]:
df_l3 = df[["lon","lat","l3"]]

df_l3.drop(df_l3.loc[df_l3["lon"].isnull()].index,inplace = True)
df_l3.drop(df_l3.loc[df_l3["lat"].isnull()].index,inplace = True)

df_test_l3 = df_l3.loc[df_l3["l3"].isnull()]

df_train_l3 = df_l3.loc[df_l3["l3"].notnull()]

In [508]:
X_l3 = df_train_l3[df_train_l3.columns.drop('l3')]
y_l3 = df_train_l3['l3']

# Partimos en entrenamiento y prueba 

for i in [1,3,5,7,9,11]:
    # Creamos el objeto del modelo
    neigh = sk.neighbors.KNeighborsClassifier(n_neighbors=i)
    
    kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
    
    scores = []
    
    for train, test in kf.split(X_l3):
        
        X_train, X_test, y_train, y_test = X_l3.iloc[train], X_l3.iloc[test], y_l3[train], y_l3[test]
    
        # Entrenamos el modelo
        neigh.fit(X_train, y_train)
    
        # Predecimos
        y_pred_l3 = neigh.predict(X_test)
    
        # Medimos la performance de la predicción
        score = sk.metrics.accuracy_score(y_test, y_pred_l3)
        scores.append(score)
        #print(score)
    print(f"n_{i} {np.mean(scores)}")


n_1 0.8291631320588276
n_3 0.8277355530873358
n_5 0.8295918114585128
n_7 0.8297374037527654
n_9 0.828520119419362
n_11 0.8274888629515317


In [510]:
X_pred_l3 = df_test_l3[df_test_l3.columns.drop('l3')]

# Entrenamiento
neigh = sk.neighbors.KNeighborsClassifier(n_neighbors=7)

# Entrenamos el modelo
neigh.fit(X_train, y_train)

# predicción
df_test_l3["l3"] = neigh.predict(X_pred_l3)

/tmp/ipykernel_17351/4193638472.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_l3["l3"] = neigh.predict(X_pred_l3)


In [511]:
df["l3"].loc[df["l3"].isnull()] = df_test_l3["l3"]

/tmp/ipykernel_17351/553463739.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["l3"].loc[df["l3"].isnull()] = df_test_l3["l3"]


In [512]:
df.drop(columns = ["ad_type","l4","l5", "l6", "created_on","start_date", "end_date","l1","l2","price_period","operation_type"], inplace = True )

In [513]:
df.dropna(subset=["price"], inplace = True)

df.dropna(subset=["currency"], inplace = True)

In [514]:
df = df.loc[df["currency"] == "USD"]

df.drop(columns = ["currency"], inplace = True )

/tmp/ipykernel_17351/3524703153.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ["currency"], inplace = True )


In [515]:
df["property_type_check"] = df.property_type.apply(lambda x: 1 if x in ("Departamento","PH") else 0)

df = df.loc[df["property_type_check"] == 1]

df.drop(columns = "property_type_check", inplace = True)

/tmp/ipykernel_17351/354315104.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["property_type_check"] = df.property_type.apply(lambda x: 1 if x in ("Departamento","PH") else 0)
/tmp/ipykernel_17351/354315104.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = "property_type_check", inplace = True)


In [516]:
df.drop(df.loc[df["rooms"].isnull() & df["bedrooms"].isnull() & df["bathrooms"].isnull() & df["surface_total"].isnull() & df["surface_covered"].isnull()].index, inplace = True)

/tmp/ipykernel_17351/766927445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df.loc[df["rooms"].isnull() & df["bedrooms"].isnull() & df["bathrooms"].isnull() & df["surface_total"].isnull() & df["surface_covered"].isnull()].index, inplace = True)


In [517]:
df_pred["rooms"].fillna(0, inplace = True)
df_pred["bedrooms"].fillna(0, inplace = True)
df_pred["bathrooms"].fillna(0, inplace = True)

In [518]:
df["rooms"].fillna(0, inplace = True)
df["bedrooms"].fillna(0, inplace = True)
df["bathrooms"].fillna(0, inplace = True)

/tmp/ipykernel_17351/764988754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rooms"].fillna(0, inplace = True)
/tmp/ipykernel_17351/764988754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bedrooms"].fillna(0, inplace = True)
/tmp/ipykernel_17351/764988754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bathrooms"].fillna(0, inplace = True)


In [519]:
df["surface_total"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0] = df["surface_covered"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0]

df["surface_total"].fillna(0, inplace = True)
df["surface_covered"].fillna(0, inplace = True)

/tmp/ipykernel_17351/3354645445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_total"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0] = df["surface_covered"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0]
/tmp/ipykernel_17351/3354645445.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_total"].fillna(0, inplace = True)
/tmp/ipykernel_17351/3354645445.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [520]:
df["bedrooms"].loc[df["bedrooms"] < 0 ] = abs(df["bedrooms"].loc[df["bedrooms"] < 0 ])

/tmp/ipykernel_17351/3611823614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bedrooms"].loc[df["bedrooms"] < 0 ] = abs(df["bedrooms"].loc[df["bedrooms"] < 0 ])


In [521]:
df["surface_total"].loc[df["surface_total"] < 0 ] = abs(df["surface_total"].loc[df["surface_total"] < 0 ])

/tmp/ipykernel_17351/3583135774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_total"].loc[df["surface_total"] < 0 ] = abs(df["surface_total"].loc[df["surface_total"] < 0 ])


In [522]:
df["surface_covered"].loc[df["surface_covered"] < 0 ] = abs(df["surface_covered"].loc[df["surface_covered"] < 0 ])

/tmp/ipykernel_17351/3604395483.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_covered"].loc[df["surface_covered"] < 0 ] = abs(df["surface_covered"].loc[df["surface_covered"] < 0 ])


In [523]:
df.drop(columns = ["title","description"], inplace = True)

/tmp/ipykernel_17351/399705157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ["title","description"], inplace = True)


In [566]:
df.drop(columns = ["lat","lon"], inplace = True)

/tmp/ipykernel_17351/1138003077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ["lat","lon"], inplace = True)


In [540]:
df.drop(df.loc[df["price"]> 6000000].index, inplace=True)

In [543]:
df.drop(df[df.price < 15000.0].index,inplace = True)

In [544]:
df.drop_duplicates(inplace = True)

In [524]:
df_pred.drop(columns = ["ad_type","l4","l5", "l6", "created_on"], inplace = True )
df_pred.drop(columns = ["l1","l2"], inplace = True )
df_pred.drop(columns = ["currency"], inplace = True )
df_pred.drop(columns = ["price_period"], inplace = True)
df_pred.drop(columns = "operation_type", inplace = True)
df_pred.drop(columns = ["start_date","end_date","services","additionals"], inplace = True)
df_pred.drop(columns = ["title","description","details"], inplace = True)

In [567]:
df_pred.drop(columns = ["lat","lon"], inplace = True)

/tmp/ipykernel_17351/3131608553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred.drop(columns = ["lat","lon"], inplace = True)


# Feature Engineering

In [553]:
df_total = pd.concat([df,df_pred], axis=0)

In [554]:
df_total["lnrooms"] = df_total.rooms.apply(lambda x: x if x == 0 else np.log(x))

In [555]:
df_total["lnbedrooms"] = df_total.bedrooms.apply(lambda x: x if x == 0 else np.log(x))

In [556]:
df_total["lnsurface_covered"] = df_total.surface_covered.apply(lambda x: x if x == 0 else np.log(x))

In [557]:
df_total["lnsurface_total"] = df_total.surface_total.apply(lambda x: x if x == 0 else np.log(x))

In [563]:
df_total["lnsurface_total"].value_counts()

0.000000    20789
3.688879     1672
3.912023     1663
3.806662     1518
3.555348     1478
            ...  
8.858653        1
3.519573        1
8.656259        1
3.832763        1
3.606856        1
Name: lnsurface_total, Length: 1109, dtype: int64

In [558]:
df_total = pd.get_dummies(df_total, columns = ["l3", "property_type"])

KeyError: "None of [Index(['l3', 'property_type'], dtype='object')] are in the [columns]"

In [559]:
df = df_total[df_total.index.isin(df.index)]
df_pred = df_total[df_total.index.isin(df_pred.index)]

# Modelización

In [569]:
X = df[df.columns.drop('price')]
y = df['price']

kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
for train, test in kf.split(X):
    X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y[train], y[test]

In [542]:
X = df[df.columns.drop('price')]
y = df['price']

for n_estimators in [3000, 4000]:
    for max_depth in [10, 12]:     

## Tienen que usar RandomForestRegressor si o si o si. Pueden cambiar los parámetros
#n_estimators = 5000
#max_depth = 5

        reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1, random_state=42)

        scores = []
        kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
        for train, test in kf.split(X):
            X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y[train], y[test]
        
            # Entrenamos el modelo
            reg.fit(X_train, y_train)
        
            # Predecimos
            y_pred = reg.predict(X_test)
        
            # Medimos la performance de la predicción
            score = sk.metrics.mean_squared_error(y_test, y_pred)
            scores.append(score)
        print(f"max_depth:{max_depth} - mean: {np.mean(scores)} sd: {np.std(scores)}")
        
    #print(f"n_{n_estimators} - mean: {np.mean(scores)} sd: {np.std(scores)}")

max_depth:10 - mean: 34920465521.80669 sd: 871275656.952177
max_depth:12 - mean: 34149354190.91881 sd: 1143433360.0695686
max_depth:10 - mean: 34916278716.37322 sd: 867722331.4316921


KeyboardInterrupt: 

## Predicción para Kaggle


In [570]:
# quedate con las mismas columnas que ar_properties
X_pred = df_pred[X.columns]

In [571]:
# los mejores hiperparámetros encontrados antes
n_estimators = 4000
max_depth = 10

# Entrenamiento
reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1)

# Entrenamos el modelo
reg.fit(X_train, y_train)

# predicción
df_pred["price"] = reg.predict(X_pred)

/tmp/ipykernel_17351/3404264150.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred["price"] = reg.predict(X_pred)


In [572]:
# Grabo la solución
df_pred[["price"]].to_csv("solucion.csv")